In [3]:
###### Scanner #######
tabelaDeSimbolos = [{"classe": "inicio", "lexema": "inicio", "tipo": "inicio"},
                    {"classe": "varinicio", "lexema": "varinicio", "tipo": "varinicio"},
                    {"classe": "varfim", "lexema": "varfim", "tipo": "varfim"},
                    {"classe": "escreva", "lexema": "escreva", "tipo": "escreva"},
                    {"classe": "leia", "lexema": "leia", "tipo": "leia"},
                    {"classe": "se", "lexema": "se", "tipo": "se"},
                    {"classe": "entao", "lexema": "entao", "tipo": "entao"},
                    {"classe": "fimse", "lexema": "fimse", "tipo": "fimse"},
                    {"classe": "fim", "lexema": "fim", "tipo": "fim"},
                    {"classe": "inteiro", "lexema": "inteiro", "tipo": "inteiro"},
                    {"classe": "literal", "lexema": "literal", "tipo": "literal"},
                    {"classe": "real", "lexema": "real", "tipo": "real"},
                   ]
allTokens = []
linha = 1
coluna = 1
char = ''
CHAR=False

fileToken = open('tokens.txt', 'w')

def getToken(file):
    
    global char,coluna,linha,allTokens

    if(char==''):
        char = file.read(1) 
        if(CHAR): print(f"CHAR1 {char}")
    token = ''
    token = scanner(file)
    #print (f"TOKEN {token}")

    while(token==None):
        char = file.read(1)
        if(CHAR): print(f"CHAR2 {char}")
        coluna+=1
        token = scanner(file)

    if(token["classe"] == "EOF"):
        return token

    if (token["classe"] != "ID" and token != token["lexema"] != ">" and token != token["lexema"] != "<" and token["classe"] != "NUM"):
            #print (token["lexema"])
            char = file.read(1)
            if(CHAR): print(f"CHAR3 {char}")
            coluna+=1
    
    naTabela = buscaTabelaDeSimbolos(tabelaDeSimbolos, token)
    
    if (naTabela != False):
        token["classe"] = naTabela["classe"]
        token["tipo"]   = naTabela["tipo"]
    elif (token["classe"] == "ID"):
            adicionarTabelaDeSimbolos(tabelaDeSimbolos, token)

    if (token["classe"] != "Comentario"):
        #print(token)
        fileToken.write(f"{token}\n")
        allTokens.append(token)
        return token
    #print("Classe: " + token["classe"] + ", Lexema: " + token["lexema"] + ", Tipo: " + token["tipo"])
        

def scanner(file):
    global char, linha, coluna
    while(char == '\t' or char == '\n' or char == ' '):
        if(char == '\n'):
            coluna = 1
            linha +=1
        else:
            coluna +=1
        char = file.read(1)
        if(CHAR): print(f"CHAR4 {char}")

    if(char == ';'):
        return {"classe" : "PT_V", "lexema": ";", "tipo":"Ponto e Vírgula"}

    if(isNum(char)):
        #print(char)
        lexema = char
        coluna +=1
        char = file.read(1)
        if(CHAR): print(f"CHAR5 {char}")
        Tipo = "inteiro"
        while(isNum(char)):
            lexema += char
            coluna +=1
            char = file.read(1)
            if(CHAR): print(f"CHAR6 {char}")
        if(char =="."):
            Tipo = "real"
            lexema += char
            coluna +=1
            char = file.read(1)
            if(CHAR): print(f"CHAR7 {char}")
            if not (isNum(char)):
                lexema+=char
                print(f"[ERRO]\tLinha{linha}:Coluna{coluna}\t{lexema} não é valido, dever tem número apois '.'")
                return None
            while(isNum(char)):
                lexema += char
                coluna +=1
                char = file.read(1)
                if(CHAR): print(f"CHAR8 {char}")
            #opicional
            if(char =="e" or char =="E"):
                lexema += char
                coluna +=1
                char = file.read(1)
                if(CHAR): print(f"CHAR9 {char}")
            
                if(char =="+" or char =="-"):
                    lexema += char
                    coluna +=1
                    char = file.read(1)
                    if(CHAR): print(f"CHAR10 {char}")
                if not (char.isnumeric()):
                    lexema+=char
                    print(f"[ERRO]\tLinha{linha}:Coluna{coluna}\t{lexema} não é valido, dever tem número apois 'E'/'e' .")
                    return None
                while(char.isnumeric()):
                    lexema += char
                    coluna +=1
                    char = file.read(1)
                    if(CHAR): print(f"CHAR11 {char}")
            while(char.isnumeric()):
                lexema += char
                coluna +=1
                char = file.read(1)
                if(CHAR): print(f"CHAR12 {char}")
        return {"classe" : "NUM", "lexema": lexema, "tipo":Tipo}

        #return token lexema tipo

    if(char == '\"'):
        lexema = ''
        coluna +=1
        char = file.read(1)
        if(CHAR): print(f"CHAR13 {char}")
        
        while(char != '\"'):
            lexema += char
            
            if(char == '\n'):
                coluna = 1
                linha +=1
            else:
                coluna +=1
            char = file.read(1)
            if(CHAR): print(f"CHAR14 {char}")
            if not char:
                lexema+=char
                print(f"[ERRO]\tLinha{linha}:Coluna{coluna}\t{lexema} não é valido, fim de arquivo sem fechar '\"'.")
                return None
        return {"classe" : "LIT", "lexema": lexema, "tipo":"Constante Literal"}
    


    if(char.isalpha()):
        lexema = ''
        while (char.isalpha() or char.isnumeric() or char =='_'):
            lexema += char
            coluna +=1
            char = file.read(1)
            if(CHAR): print(f"CHAR15 {char}")
        return {"classe" : "ID", "lexema" : lexema, "tipo" : "nulo"}
        #return token lexema tipo

    
    if(char == '{'):
        lexema = char
        while(char != '}'):
            char = file.read(1)
            if(CHAR): print(f"CHAR16 {char}")
            if(char == '\n'):
                coluna = 1
                linha +=1
            else:
                coluna +=1
            lexema += char
            if not char:
                lexema+=char
                print(f"[ERRO]\tLinha{linha}:Coluna{coluna}\t{lexema} não é valido, fim de arquivo sem fechar '}}'.")
                return None
        return {"classe" : "Comentario", "lexema": lexema, "tipo":"Comentario"}

    if not char:
        return {"classe" : "EOF", "lexema": "EOF", "tipo":"EOF"}



    if(char == '<'):
        lexema = char
        char = file.read(1)
        if(CHAR): print(f"CHAR17 {char}")
        coluna+=1
        if(char == '>'):
            lexema += char
            return {"classe" : "OPR", "lexema": lexema, "tipo":"Operador relacional"}
        elif(char == '='):
            lexema += char
            return {"classe" : "OPR", "lexema": lexema, "tipo":"Operador relacional"}
        elif(char == '-'):
            lexema += char
            return {"classe" : "ATR", "lexema": lexema, "tipo":"Atribuição"}
        else:
            return {"classe" : "OPR", "lexema": "<", "tipo":"Operador relacional"}

    if(char == '='):
        return {"classe" : "OPR", "lexema": "=", "tipo":"Operador relacional"}

    if(char == '>'):
        char = file.read(1)
        if(CHAR): print(f"CHAR18 {char}")
        coluna+=1
        if(char == '='):
            return {"classe" : "OPR", "lexema": ">=", "tipo":"Operador relacional"}
        elif(char == '\t' or char == '\n' or char == ' ' or not char): #not char necessário ?
            return {"classe" : "OPR", "lexema": ">", "tipo":"Operador relacional"}
        else:
            return {"classe" : "OPR", "lexema": ">", "tipo":"Operador relacional"}

    if(char == '+'):
        return {"classe" : "OPA", "lexema": "+", "tipo":"Operador aritmético"}
    if(char == '-'):
        return {"classe" : "OPA", "lexema": "-", "tipo":"Operador aritmético"}
    if(char == '*'):
        return {"classe" : "OPA", "lexema": "*", "tipo":"Operador aritmético"}
    if(char == '/'):
        return {"classe" : "OPA", "lexema": "/", "tipo":"Operador aritmético"}


    if(char == '('):
        return {"classe" : "AB_P", "lexema": "(", "tipo":"Abre Parênteses"}

    if(char == ')'):
        return {"classe" : "FC_P", "lexema": ")", "tipo":"Fecha Parênteses"}



    if(char == ','):
        return {"classe" : "VIR", "lexema": ",", "tipo":"Vírgula"}
    
    print(f"[ERRO]\tLinha{linha}:Coluna{coluna}\tCaracter '{char}' não é valido.") 

    return(None)
      
def buscaTabelaDeSimbolos(tabelaDeSimbolos, token):

    for x in tabelaDeSimbolos:
        if x["lexema"] == token["lexema"]:
            return x

    return False

def adicionarTabelaDeSimbolos(tabelaDeSimbolos, token):
    tabelaDeSimbolos.append(token)

def isNum(num):
    if(num == '0' or num == '1' or num == '2' or num == '3' or num == '4' or num == '5' or num == '6' or num == '5' or num == '8' or num == '9'):
        return True
    else:
        return False

def getLinhaColuna():
    global linha,coluna

    return f"Linha[{linha}] : Colinha[{coluna}]"

def returnTabeladesimbulo():
    global tabelaDeSimbolos
    print(tabelaDeSimbolos)

def modTabelaDeSimbolosTipo(lexema,tipo):
    global tabelaDeSimbolos
    for x in tabelaDeSimbolos:
        
        if x["lexema"] == lexema:
            x["tipo"] = tipo
            return True
    return False

def returnTipoDotoken(lexema):
    global allTokens
    for x in allTokens:
        
        if x["lexema"] == lexema:
            return x["tipo"]
    return False

########### Parcer #############
import pandas as pd

PRINT_PILHA = False
GET_ON_TABLE = False
REDUCTION_GRAM = True
REDUCE = False
SHIFT = False
TOKEN = False
FULL_PILHA = False
LEXEMA = False
gram = [
    ["P'", "P"],                #1
    ["P","inicio","V","A"],     #2
    ["V","varinicio","LV"],     #3
    ["LV","D","LV"],            #4
    ["LV","varfim","PT_V"],     #5
    ["D","TIPO","L","PT_V"],    #6
    ["L","ID","VIR","L"],       #7
    ["L","ID"],                 #8
    ["TIPO","inteiro"],         #9
    ["TIPO","real"],            #10
    ["TIPO","literal"],         #11
    ["A","ES","A"],             #12
    ["ES","leia","ID","PT_V"],  #13
    ["ES","escreva","ARG","PT_V"],#14
    ["ARG","LIT"],              #15
    ["ARG","NUM"],              #16
    ["ARG","ID"],               #17
    ["A","CMD","A"],            #18
    ["CMD","ID","ATR","LD","PT_V"],#19
    ["LD","OPRD","OPA","OPRD"],#20
    ["LD","OPRD"],              #21
    ["OPRD","ID"],              #22
    ["OPRD","NUM"],             #23
    ["A","COND","A"],           #24
    ["COND","CAB","CP"],        #25
    ["CAB","se","AB_P", "EXP_R","FC_P","entao"],#26
    ["EXP_R","OPRD","OPR","OPRD"],#27
    ["CP","ES","CP"],           #28
    ["CP","CMD","CP"],          #29
    ["CP","COND","CP"],         #30
    ["CP","fimse"],             #31
    ["A","fim"]                 #32
]
pilha = [0]

def main():

    iniciaObj()
    global pilha, gram
    Tabela = pd.read_csv("Tabela1.csv")
    file = open('code.txt', 'r')
    #scan(file)

    token = getToken(file)
    if(TOKEN):
        print(f"TOKEN {token}")
    #print(Tabela[["inteiro"]])

    #print(Tabela)
    while(True):
        
        if((token["classe"]=="EOF" and pilha[-1] == "0")):
            break


        while(token == None):#comentario
            token = getToken(file)
            if(TOKEN):
                print(f"TOKEN {token}")

        UltimoPilha = pilha.pop()
        
        #print(Tabela.loc[2,"varinicio"])
        if(GET_ON_TABLE):
            print(f"GET TABELA1 [{UltimoPilha}] , [{token['classe']}] = {Tabela.loc[int(UltimoPilha),token['classe']]} ")
        
        Action = Tabela.loc[int(UltimoPilha),token['classe']]
        while pd.isnull(Tabela.loc[int(UltimoPilha),token['classe']]):
            
            print(f"[ERRO_PARSER]\t{getLinhaColuna()} -> SRL TABLE[{UltimoPilha}][{token['classe']}] = {Action}, A sintaxe não valida, ferifique o Token: {token}" )
            token = getToken(file)
            if(GET_ON_TABLE):
                print(f"GET TABELA1 [{UltimoPilha}] , [{token['classe']}] = {Tabela.loc[int(UltimoPilha),token['classe']]} ")
            Action = Tabela.loc[int(UltimoPilha),token['classe']]

        if Action == "acc":
            break
        #print(Action)
        
        
        Action = Action.split(".")
        #print (f"Action {Action}")
        #print(Action)
        
        while(Action[0] == 'R'):
            if(REDUCE):
                print(f"-->\t\tReduce {Action[1]}")

            Gram = gram[int(Action[1])]
            
            if(REDUCTION_GRAM):
                print(f">>>>>>>>>>>>>>>>> Gram[{int(Action[1])+1}] - {Gram[0]} ->",end=' ')
                for i in (range(1,len(Gram))):
                    print(f"{Gram[i]}",end=' ')
                print()
                
            
            lexemaList = []
            lexemaListTemp = []
            for i in reversed(range(1,len(Gram))):
                UltimoPilha = pilha.pop() # token
                lexemaListTemp.append(UltimoPilha['lexema'])

                if((UltimoPilha['classe'] != Gram[i]) == True): 
                    print(UltimoPilha['classe'] != Gram[i])
                    print(f"{UltimoPilha['classe']}!={Gram[i]}.")
                if(UltimoPilha['classe'] != Gram[i]):#!!!!!!!
                    print("[ERRO]\tTabela e gramatica não bate.")
                UltimoPilha = pilha.pop() # num
            lexema = ''
            
            for i in reversed(range(1,len(Gram))):
                lexemaList.append(lexemaListTemp.pop())

            #tempList = lexemaList.copy()
            tempList = []
            #tempList = [item for sublist in lexemaList for item in sublist]
            for A in lexemaList:
                if (type(A) == list):
                    for item in A:
                        tempList.append(item)
                else: 
                    tempList.append(A)

            #print(f"templist{tempList}")

            if(LEXEMA):
                print(tempList)
                print()
#################### OBJ
            makeObj(int(Action[1])+1,tempList)
            #print(tempList)

            

            pilha.append(UltimoPilha)
            NewToken = {"classe" : Gram[0], "lexema": tempList, "tipo":'?'} 
            pilha.append(NewToken)
            if(GET_ON_TABLE):
                print(f"GET TABELA2 [{int(UltimoPilha)}] , [{Gram[0]} ] = {Tabela.loc[int(UltimoPilha),Gram[0]]}")
            
            pilha.append(Tabela.loc[int(UltimoPilha),Gram[0]])
            
            if(PRINT_PILHA):
                print(pilha)

            UltimoPilha = pilha.pop()
            Action = Tabela.loc[int(UltimoPilha),token['classe']]

            while pd.isnull(Tabela.loc[int(UltimoPilha),token['classe']]):
                print(f"[ERRO_PARSER]\t{getLinhaColuna()} -> SRL TABLE[{UltimoPilha}][{token['classe']}] = {Action}, A sintaxe não valida, verifique o Token: {token}" )
                token = getToken(file)
                if(GET_ON_TABLE):
                    print(f"GET TABELA1 [{UltimoPilha}] , [{token['classe']}] = {Tabela.loc[int(UltimoPilha),token['classe']]} ")
                Action = Tabela.loc[int(UltimoPilha),token['classe']]

            if Action == "acc":

                break
            if(GET_ON_TABLE):
                print(f"GET TABELA3 [{UltimoPilha}] , [{token['classe'] }] = {Action}")
            Action = Action.split(".")
            if(GET_ON_TABLE):
                print(f"ACTION{Action}")
            
        
        if(Action[0] == 'S'):
            if(SHIFT):
                print(f"-->\t\tShift {Action[1]}")
            #print(f"UltimoPilha = {UltimoPilha}")
            pilha.append(UltimoPilha)
            pilha.append(token)
            pilha.append(Action[1])

        if Action == "acc":
            break
        if(Action[0] != 'S' and Action[0] != 'R'):
            print(f"[ERRO_PARSER]\t{getLinhaColuna()}\t{Action} nao valida" )
            pilha.append(UltimoPilha)

        token = getToken(file)
        if(TOKEN):
            print(f"TOKEN {token}")
        if(FULL_PILHA):
            print()
            print(*pilha, sep='\n')
            print()
        


########### Objeto ###########


varInt  = []
varDouble  = []
varLit = []
tempInt  = []
tempDouble  = []
tempLit = []
ARGfix = []
ARG = []
Cont = 0
tab = 0
def listVar():
    global varInt, varDouble, varLit, tempInt
    print(f"int{varInt}")
    print(f"double{varDouble}")
    print(f"lit{varLit}")
    print(f"temp int{tempInt}")
    print(f"temp double{tempDouble}")



def iniciaObj():
    file = open('code.c', 'a')
    file.write("#include<stdio.h>\ntypedef char literal[256];\nvoid main(void){\n\n\n\n")



def makeObj(gramNum,token):

    global varInt, varDouble, varLit, ARG, Cont, tab, tempDouble, tempLit
    file = open('code.c', 'a')

    if(gramNum==5):
        file.write("\n")

    if(gramNum==6): ############  A
        file.write(f";\n")
        print (token)
    #if False:
        if (token[0] == "literal"):
            #print("lit")
            for i in (range(1,len(token),2)):
                varLit.append(token[i])
                modTabelaDeSimbolosTipo(token[i],"literal")
                #print(f"test lit[{token[i]}]")
        elif (token[0] == "real"):
            #print("real")
            for i in (range(1,len(token),2)):
                varDouble.append(token[i])
                modTabelaDeSimbolosTipo(token[i],"real")
                #print(token[i])
        elif (token[0] == "inteiro"):
            #print("int")
            for i in (range(1,len(token),2)):
                varInt.append(token[i])
                modTabelaDeSimbolosTipo(token[i],"inteiro")
                #print(token[i])
        else:
            print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} Tipo {token[0]} não conhecido" )
        
    
    if(gramNum==7): ############  B 
       # L→ id vir L

        file.write(f", {token[0]}")

    if(gramNum==8): ############  C
        print(token)
        # L→ id
        file.write(f"\t{token[0]}")

    if(gramNum==9):
        
        file.write("\tint ")

    if(gramNum==10):
        
        file.write("\tdouble ")

    if(gramNum==11):
        #print ( token)
        file.write(f"\t{token[0]}")#literal
    
    if(gramNum==13):
        
        if(token[1] in varInt):
            file.write(f"\tscanf(\"%d\",&{token[1]});\n")
        elif (token[1] in varDouble):
            file.write(f"\tscanf(\"%lf\",&{token[1]});\n")
        elif(token[1] in varLit):
            file.write(f"\tscanf(\"%s\",{token[1]});\n")
        else:
            print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} - Variável [{token[1]}] não conhecido" )

    if(gramNum==14):####### arrumar bug print variavel
        #print (token)
        #print (ARG)
        for x in range(0,tab):
            file.write("\t")
        for x in (range(0,len(ARG))):
            if(ARG[x][1] == token[1]):
                if(ARG[x][0] == "id"):
                    if(ARG[x][1] in varInt):
                        file.write(f"\tprintf(\"%d\",{ARG[x][1]});\n")
                    elif(ARG[x][1] in varDouble):
                        file.write(f"\tprintf(\"%lf\",{ARG[x][1]});\n")
                    elif(token[1] in varLit):
                        file.write(f"\tprintf(\"%s\",{ARG[x][1]});\n")
                elif(ARG[x][0] == "num"):
                    file.write(f"\tprintf(\"%ld\",{ARG[x][1]});\n")
                elif(ARG[x][0] == "literal"):
                    file.write(f"\tprintf(\"{token[1]}\");\n") 
                else:
                    print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} - Variável [{token[1]}] não conhecido" )
                ARG.pop()

    if(gramNum==15): #lit
        #print(token)
        temp = ['literal', token[0]]
        ARG.append(temp)

    if(gramNum==16): #num (real ou int)
        temp = ['num', token[0]]
        ARG.append(temp)
    
    if(gramNum==17): #id (variavel)

        if not((token[0] in varInt) or (token[0] in varDouble) or (token[0] in varLit)):
            print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} - Variável [{token[0]}] não declarada")
        else:
            temp = ['id', token[0]]
            ARG.append(temp)


    if(gramNum==19): 
        #print(token)

        vars = []
        
        for x in range(0,len(token),2):
            temp = returnTipoDotoken(token[x])
            if(temp == "nulo"):
                if (token[x] in varInt or token[x] in tempInt): 
                    vars.append("int")
                elif (token[x] in varDouble or token[x] in tempDouble):
                    vars.append("double")
                elif (token[x] in varLit):
                    vars.append("lit")
                    print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} - Não é possivel operar variavel literal")
                else: 
                    print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} - ERRO INESPERADO - CODIGO 19 - token[x] = [{token[x]}]")
            elif (temp == False): 
                print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} - ERRO INESPERADO - CODIGO 19 - Função returnTipoDotoken retornou Falso")
            else: 
                if(temp == "inteiro"): vars.append("int")
                elif (temp == "real"): vars.append("double")
                else: print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} - ERRO INESPERADO - CODIGO 19 - Variavel [{token[x]}] = {temp}, não e possivel operar")
        
        ERRO = False
        
        for i in range(1,len(vars)):
            if(vars[0] != vars[i]):
                ERRO = True
        
        if(ERRO):
            msgErro = ''
            for x, i in zip(range(0,len(token),2), range(0,len(vars))):
                msgErro += f"Variavel [{token[x]}] é um [{vars[i]}]; "

            print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} - Tipos diferentes para atribuição -> {msgErro}")
        else:
            for x in range(0,tab):
                file.write("\t")
            file.write(f"\t{token[0]} = ")
            if(len(token) > 4 ):
                file.write(f"T{Cont-1};\n")
            else:
                file.write(f"{token[2]};\n")

    if(gramNum==20): 
        # LD→ OPRD opa OPRD
        #print (token)
        vars = []
        for x in range(0,tab):
            file.write("\t")
        for x in range(0,len(token),2):
            temp = returnTipoDotoken(token[x])
            if(temp == "nulo"):
                if (token[x] in varInt): vars.append("int")
                elif (token[x] in varDouble): 
                    vars.append("double")
                elif (token[x] in varDouble): 
                    vars.append("lit") 
                    print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} - Não é possivel operar variavel literal")
                else: 
                    print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} - ERRO INESPERADO - CODIGO 20 - token[x] = [{token[x]}]")
            elif (temp == False): print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} - ERRO INESPERADO - CODIGO 20 - Função returnTipoDotoken retornou Falso")
            else: 
                if(temp == "inteiro"): 
                    vars.append("int")
                elif (temp == "real"): 
                    vars.append("double")
                elif (temp == "literal"):
                    print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} - Não é possivel operar variavel literal")
                else: print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} - ERRO INESPERADO - CODIGO 20 - Variavel [{token[x]}] = {temp}, não e possivel operar")
        
        ERRO = False
       
        for i in range(1,len(vars)):
            if(vars[0] != vars[i]):
                ERRO = True
        
        if(ERRO):
            msgErro = ''
            for x, i in zip(range(0,len(token),2), range(0,len(vars))):
                msgErro += f"Variavel [{token[x]}] é um [{vars[i]}]; "

            print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} - Tipos diferentes para atribuição -> {msgErro}")
        else:
            if(vars[0] == "int"):
                tempInt.append(f"T{Cont}")
            if(vars[0] == "double"):
                tempDouble.append(f"T{Cont}")

            file.write(f"\tT{Cont} = ")
            for x in range(0,len(token)):
                file.write(f"{token[x]} ")
            file.write(";\n")
            Cont+=1

    if(gramNum==21):
        #print (token)
        file.write("") # nao tem nessecidade, ja e feito no 27

    if(gramNum==22): 
        #print(token)

        if not((token[0] in varInt) or (token[0] in varDouble)):
            print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} - Variável [{token[0]}] não declarada")
        
    
    if(gramNum==23): # nao tem nessecidade, ja e feito no 27
        file.write("")

    if(gramNum==25):
        for x in range(0,tab):
            file.write("\t")
        file.write("\t}\n")

    if(gramNum==26):
        for x in range(0,tab):
            file.write("\t")
        file.write("\tif(")
           
        file.write(f"T{Cont-1})")
        file.write("{\n")
        tab+=1
    if(gramNum==27): # fazer erro
        #print(token)

        vars = []
        for x in range(0,tab):
            file.write("\t")
        for x in range(0,len(token),2):
            temp = returnTipoDotoken(token[x])
            if(temp == "nulo"):
                if (token[x] in varInt):
                    vars.append("int")
                elif (token[x] in varDouble): 
                    vars.append("double")
                elif (token[x] in varDouble): 
                    vars.append("lit") 
                    print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} - Não é possivel operar variavel literal")
                else: 
                    print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} - ERRO INESPERADO - CODIGO 27 - token[x] = [{token[x]}]")
            elif (temp == False): print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} - ERRO INESPERADO - CODIGO 27 - Função returnTipoDotoken retornou Falso")
            else: 
                if(temp == "inteiro"): 
                    vars.append("int")
                elif (temp == "real"): 
                    vars.append("double")
                elif (temp == "literal"):
                    print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} - Não é possivel operar variavel literal")
                else: 
                    print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} - ERRO INESPERADO - CODIGO 27 - Variavel [{token[x]}] = {temp}, não e possivel operar")
        ERRO = False
       
        for i in range(1,len(vars)):
            if(vars[0] != vars[i]):
                ERRO = True
        
        if(ERRO):
            msgErro = ''
            for x, i in zip(range(0,len(token),2), range(0,len(vars))):
                msgErro += f"Variavel [{token[x]}] é um [{vars[i]}]; "

            print(f"[ERRO_LEXEMA]\t{getLinhaColuna()} - Tipos diferentes para atribuição -> {msgErro}")
        else:
            tempInt.append(f"T{Cont}")

            file.write(f"\tT{Cont} = ")
            for x in range(0,len(token)):
                file.write(f"{token[x]} ")
            file.write(";\n")
            Cont+=1

    if(gramNum==31): ############  D
        tab-=1

    if(gramNum==32): ############  E
        file.write("}")
        file.close()
        colocarAsTemp()


def colocarAsTemp():
    with open('code.c','r',encoding='utf-8') as file:
        data = file.readlines()
    addTemp = "\t/*----Variaveis temporarias----*/\n"
    if(len(tempInt)>0):
        addTemp+="\tint"
        for x in range(0,len(tempInt)-1):
            addTemp+=f" {tempInt[x]},"
        addTemp+=f" {tempInt[len(tempInt)-1]}"
        addTemp+=";\n"
    if(len(tempDouble)>0):
        addTemp+="\tdouble"
        for x in range(0,len(tempDouble)-1):
            addTemp+=f" {tempDouble[x]},"
        addTemp+=f" {tempDouble[len(tempDouble)-1]}"
        addTemp+=";\n"
    if(len(tempLit)>0):
        addTemp+="\tliteral"
        for x in range(0,len(tempLit)-1):
            addTemp+=f" {tempLit[x]},"
        addTemp+=f" {tempLit[len(tempLit)-1]}"
        addTemp+=";\n"
    addTemp+="\t/*------------------------------*/"
    data[4] = addTemp
    file.close()

    with open('code.c', 'w', encoding='utf-8') as file:
        file.writelines(data)

  
if __name__ == "__main__":
    main()
    listVar()
    #returnTabeladesimbulo()
    #print(pilha)



>>>>>>>>>>>>>>>>> Gram[11] - TIPO -> literal 
>>>>>>>>>>>>>>>>> Gram[8] - L -> ID 
['A']
>>>>>>>>>>>>>>>>> Gram[6] - D -> TIPO L PT_V 
['literal', 'A', ';']
>>>>>>>>>>>>>>>>> Gram[9] - TIPO -> inteiro 
>>>>>>>>>>>>>>>>> Gram[8] - L -> ID 
['D']
>>>>>>>>>>>>>>>>> Gram[7] - L -> ID VIR L 
>>>>>>>>>>>>>>>>> Gram[6] - D -> TIPO L PT_V 
['inteiro', 'B', ',', 'D', ';']
>>>>>>>>>>>>>>>>> Gram[10] - TIPO -> real 
>>>>>>>>>>>>>>>>> Gram[8] - L -> ID 
['C']
>>>>>>>>>>>>>>>>> Gram[6] - D -> TIPO L PT_V 
['real', 'C', ';']
>>>>>>>>>>>>>>>>> Gram[5] - LV -> varfim PT_V 
>>>>>>>>>>>>>>>>> Gram[4] - LV -> D LV 
>>>>>>>>>>>>>>>>> Gram[4] - LV -> D LV 
>>>>>>>>>>>>>>>>> Gram[4] - LV -> D LV 
>>>>>>>>>>>>>>>>> Gram[3] - V -> varinicio LV 
>>>>>>>>>>>>>>>>> Gram[15] - ARG -> LIT 
>>>>>>>>>>>>>>>>> Gram[14] - ES -> escreva ARG PT_V 
>>>>>>>>>>>>>>>>> Gram[13] - ES -> leia ID PT_V 
>>>>>>>>>>>>>>>>> Gram[15] - ARG -> LIT 
>>>>>>>>>>>>>>>>> Gram[14] - ES -> escreva ARG PT_V 
>>>>>>>>>>>>>>>>> Gram[13] - ES 